In [ ]:
""" 
Author: p.wagner@bhvi.org / p.wagner@unsw.edu.au 
image registration of anterior segment projection from several scans 

Purpose: 

statistical analyese of choroid thickness maps after intra and inter participant alignment


"""
import pandas as pd
import numpy as np
import os.path
from pathlib import Path
from pystackreg import StackReg
from skimage import io
from scipy import stats

import natsort
import sys
sys.path.append(r'C:\Users\p.wagner\Documents\Python Scripts\oct_data_analyses_helpers')
from oct_helpers_lib import OctDataAccess as get_px_meta
from oct_helpers_lib import TopconSegmentationData as TSD

import cv2
import glob
from PIL import Image
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px

path_oct = r'E:\studyIII\OCT_data'
path_logbook = r'C:\Users\p.wagner\Documents\phd\C4 study_III\participants'
fn_logbook = 'participant_log_studyIII_V0.2.xlsx' 
fp_fn_logbook = os.path.join(path_logbook, fn_logbook)
thickness_csv = 'DEFAULT_3D_All_Thickness.csv'


# check if path_oct is available 
if not os.path.isdir(path_oct):
    print('OCT data path NOT available')

In [ ]:
residuals_data_fp = r'E:\studyIII\OCT_final_data'
def get_choroid_thickness_data(px_ids, eye_id):
    # raw data
#     residuals_fn = str(px_id) +'_' + eye_id + '_choroid_thickness_map_mm_raw.csv'
    # filtered data 
#     residuals_fn = str(px_id) +'_' + eye_id + '_choroid_thickness_map_mm_std_filter.csv'
    nan_counter = []
    layer_all = []   
    for px_id in px_ids:
        residuals_fn = str(px_id) +'_' + eye_id + '_choroid_thickness_map_mm_std_filter.csv'
        fpn = os.path.join(residuals_data_fp, residuals_fn)
#         print(fpn)
        r_map = pd.read_csv(fpn, index_col=0).values.astype(float) 
        layer_all.append(r_map)
        
        # count all pixel positions with NO depth data 
        idxs = np.where(np.isnan(r_map)) 
        mask = np.zeros(r_map.shape)
        mask[idxs[0], idxs[1]] = 1
        nan_counter.append(mask)
        
    layer_all = np.array(layer_all)    
    nan_counter = np.nansum(np.array(nan_counter), axis=0)
    
    return layer_all, nan_counter

def map_significant_data(display_data, save_file=False, output_fpn=None):
    # creating a accumulative map of dioptric landscape
    fig = px.imshow(display_data,
                    title=title_name,
                    labels=dict(x='Horizontal decentration from fovea in degrees [°]',
                                y='Vertical decentration from fovea in degrees [°]',
                                color='Thickness change [μm]',
                                ),
#                     # pixels to degrees translation
                    x=np.array([float(x) * 0.0703 for x in range(0, 512)]) - 18,
                    y=-1 * np.array([float(x) * 0.10547 for x in range(0, 256)]) + 13.5,
#                         y = 1 * np.array([int(x) for x in range(0, 256)])
                    )
    fig.update_xaxes(side="bottom")
    fig.update_yaxes(autorange=True)

    fig.update_coloraxes(cmid=0,
#                          cmin=-30,
#                          cmax= 30,
                         colorscale='Portland',

#                           colorscale=[[0.0, "rgb(255,  0,  0)"],
#                                      [0.05, "rgb(255,  0,  0)"],
#                                      [0.05, "rgb(125,125,  0)"],
#                                      [0.1,  "rgb(125,125,  0)"],
#                                      [0.1,  "rgb(255,255,255)"],
# #                                      [0.2,  "rgb(200,125,100)"],
# #                                      [0.2,  "rgb(200,200,100)"],
# #                                      [0.3,  "rgb(200,200,100)"],
# #                                      [0.3,  "rgb(200,200,200)"],
# #                                      [0.5,  "rgb(200,200,200)"],
#                                      [1.0,  "rgb(155,155,155)"]],
                        colorbar_title_side='right',
                        colorbar_tickfont=dict(size=18, ),
                        colorbar_title_font=dict(size=18, ),
                        reversescale=False,
                         )

    fig.update_layout(title_font_size=24,
                      autosize=True,
                      width=1200, height=900,
                      margin=dict(l=10, r=10, b=10, t=40),
                      )
    fig.update_xaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )
    fig.update_yaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )

    if save_file:
        print('saved ', output_fpn)
#         fig.write_image(output_fpn)
    else:
        fig.show()

def map_std_data(display_data, save_file=False, output_fpn=None):
    # creating a accumulative map of dioptric landscape
    fig = px.imshow(display_data,
                    title=title_name,
                    labels=dict(x='Horizontal decentration from fovea in degrees [°]',
                                y='Vertical decentration from fovea in degrees [°]',
                                color='Standard deviation [μm]',
                                ),
#                     # pixels to degrees translation
                    x=np.array([float(x) * 0.0703 for x in range(0, 512)]) - 18,
                    y=-1 * np.array([float(x) * 0.10547 for x in range(0, 256)]) + 13.5,
#                         y = 1 * np.array([int(x) for x in range(0, 256)])
                    )
    fig.update_xaxes(side="bottom")
    fig.update_yaxes(autorange=True)

    fig.update_coloraxes(
                        colorscale='YlOrRd',

                        colorbar_title_side='right',
                        colorbar_tickfont=dict(size=18, ),
                        colorbar_title_font=dict(size=18, ),
                        reversescale=False,
                         )

    fig.update_layout(title_font_size=24,
                      autosize=True,
                      width=1200, height=900,
                      margin=dict(l=10, r=10, b=10, t=40),
                      )
    fig.update_xaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )
    fig.update_yaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )

    if save_file:
        print('saved ', output_fpn)
#         fig.write_image(output_fpn)
    else:
        fig.show()
        
def map_point_inclusion_data(display_data, save_file=False, output_fpn=None):
    # creating a accumulative map of dioptric landscape
    fig = px.imshow(display_data,
                    title=title_name,
                    labels=dict(x='Horizontal decentration from fovea in degrees [°]',
                                y='Vertical decentration from fovea in degrees [°]',
                                color='n number of valid data per pixel positon',
                                ),
#                     # pixels to degrees translation
                    x=np.array([float(x) * 0.0703 for x in range(0, 512)]) - 18,
                    y=-1 * np.array([float(x) * 0.10547 for x in range(0, 256)]) + 13.5,
#                         y = 1 * np.array([int(x) for x in range(0, 256)])
                    )
    fig.update_xaxes(side="bottom")
    fig.update_yaxes(autorange=True)

    fig.update_coloraxes(
                        colorscale='cividis',

                        colorbar_title_side='right',
                        colorbar_tickfont=dict(size=18, ),
                        colorbar_title_font=dict(size=18, ),
                        reversescale=False,
                         )

    fig.update_layout(title_font_size=24,
                      autosize=True,
                      width=1200, height=900,
                      margin=dict(l=10, r=10, b=10, t=40),
                      )
    fig.update_xaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )
    fig.update_yaxes(title_font=dict(size=22, ), tickfont=dict(size=18), )

    if save_file:
        print('saved ', output_fpn)
#         fig.write_image(output_fpn)
    else:
        fig.show()
        
        
def appl_stats_on_map(map1):
    p_values = np.empty((256, 512), dtype=float) 

    for row_idx in range(0,255): 
        for col_idx in range(0,511):
            array1 = map1[:, row_idx, col_idx]
#             array2 = map2[:, row_idx, col_idx]

            p_values[row_idx, col_idx] = (stats.wilcoxon(array1, y=None)[1])
    return p_values

def map_pValues(display_data, save_file=False, output_fpn=None):
    # creating a accumulative map of dioptric landscape
    fig = px.imshow(display_data,
                    title=title_name,
                    labels=dict(x='Horizontal decentration from fovea in degrees [°]',
                                y='Vertical decentration from fovea in degrees [°]',
                                color="p-Values",
                                ),
#                     # pixels to degrees translation
                    x=np.array([float(x) * 0.0703 for x in range(0, 512)]) - 18,
                    y=-1 * np.array([float(x) * 0.10547 for x in range(0, 256)]) + 13.5,
#                         y = 1 * np.array([int(x) for x in range(0, 256)])
                    )
    fig.update_xaxes(side="bottom")
    fig.update_yaxes(autorange=True)

    fig.update_coloraxes(
                          colorscale=[[0.0, "rgb(255,  0,  0)"],
                                     [0.05, "rgb(255,  0,  0)"],
                                     [0.05, "rgb(125,125,  0)"],
                                     [0.1,  "rgb(125,125,  0)"],
                                     [0.1,  "rgb(255,255,255)"],
#                                      [0.2,  "rgb(200,125,100)"],
#                                      [0.2,  "rgb(200,200,100)"],
#                                      [0.3,  "rgb(200,200,100)"],
#                                      [0.3,  "rgb(200,200,200)"],
#                                      [0.5,  "rgb(200,200,200)"],
                                     [1.0,  "rgb(155,155,155)"]],
                        colorbar_title_side='right',
                        colorbar_tickfont=dict(size=18, ),
                        colorbar_title_font=dict(size=18, ),
                        reversescale=False
                         )

    fig.update_layout(title_font_size=30,
                      autosize=True,
                      width=1200, height=900,
                      margin=dict(l=10, r=10, b=10, t=50),
                      )
    fig.update_xaxes(title_font=dict(size=22, ), tickfont=dict(size=28), )
    fig.update_yaxes(title_font=dict(size=22, ), tickfont=dict(size=28), )

    if save_file:
        fig.write_image(output_fpn)
    else:
        fig.show()

In [ ]:
px_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 23, 26, ] # all px
# px_ids = [1,    3,    5, 6, 7, 8,    10, 11, 12, 13, 14, 15,     17, 18, 20, 23, 26, ] # worstest removed 
# px_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,     17, 18, 20, 23, 26, ] # 16 removed 
eye_id = 'OS'

r_maps, nan_map = get_choroid_thickness_data(px_ids, eye_id)

# creat images from residal choroid thickness 
output_fp = r'E:\studyIII\OCT_data\images\residuals'

# # for control purpose only map source data of choroidal thickness residual maps 
# for idx, px_id in enumerate(px_ids):
#     output_fn = str(px_id) + '_'+ eye_id + '_choroid_thickness_residual_pre_post.png'
#     output_fpn = os.path.join(output_fp, output_fn)
#     display_data = r_maps[idx, :,:]
    
#     title_name = ('Choroid thickness residual: '+ eye_id + ' , px_id: ' + str(px_id) +
#                   ', avg: ' + str(np.round(np.nanmean(display_data), 2)) +
#                   ', std: ' + str(np.round(np.nanstd(display_data), 2))  +
#                   ', max: ' + str(np.round(np.nanmax(display_data), 2)) + 
#                   ', min: ' + str(np.round(np.nanmin(display_data), 2))        
#                  ) 
#     map_significant_data(display_data, save_file=True, output_fpn=output_fpn)


In [ ]:
# display_data = nan_map
# display_data[display_data>10] = 10  
# fig = go.Figure(data=[go.Surface(z=display_data)])


# fig.update_layout(scene_camera_eye=dict(y=-1, x=0.01, z=12),
#                   scene_aspectmode='manual', 
#                   scene_aspectratio=dict(x=12, y=9, z=10),
#                   scene = dict(xaxis = dict(nticks=10, range=[0,512],),
#                                yaxis = dict(nticks=10, range=[0,256],),
#                                zaxis = dict(nticks=10, range=[-30,30],),),)

# fig.show()

In [ ]:
display_data = np.abs(nan_map - np.max(nan_map))
display_data[display_data <= 10] = 10
title_name = ('Valid data per pixel position '+ eye_id + ', of available data:'
                  ' avg: ' + str(np.round(np.nanmean(display_data), 2)) +
                  ', std: ' + str(np.round(np.nanstd(display_data), 2))
             )



map_point_inclusion_data(display_data, save_file=False, output_fpn=None)

In [ ]:
max_exclusion = 6
display_data = np.nanmean(r_maps, axis=0)

idxs = np.where(nan_map>=max_exclusion)
display_data[idxs] = np.nan
idxs = np.where(nan_map>=max_exclusion)
display_data[idxs] = np.nan

title_name = ('Mean choroid thickness change, ' + eye_id +
              ', avg: ' + str(np.round(np.nanmean(display_data), 2)) +
              ', std: ' + str(np.round(np.nanstd(display_data), 2))  +
              ', max: ' + str(np.round(np.nanmax(display_data), 2)) + 
              ', min: ' + str(np.round(np.nanmin(display_data), 2))        
             ) 
map_significant_data(display_data, save_file=False, output_fpn=None)
fig_hist =plt.hist(display_data.flatten(), 100)

In [ ]:
max_exclusion = 6
display_data = np.nanstd(r_maps, axis=0)

idxs = np.where(nan_map>=max_exclusion)
display_data[idxs] = np.nan
idxs = np.where(nan_map>=max_exclusion)
display_data[idxs] = np.nan

title_name = ('Standard deviation of residual, ' + eye_id +
              ', avg: ' + str(np.round(np.nanmean(display_data), 2)) +
              ', std: ' + str(np.round(np.nanstd(display_data), 2))  +
              ', max: ' + str(np.round(np.nanmax(display_data), 2)) + 
              ', min: ' + str(np.round(np.nanmin(display_data), 2))        
             )

map_std_data(display_data, save_file=False, output_fpn=None)

In [ ]:
p_values = appl_stats_on_map(r_maps)

In [ ]:


display_data = p_values

mask = np.ones((256, 512))
idxs = np.where(nan_map>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(nan_map>=max_exclusion)
mask[idxs[0], idxs[1]] = 0
idxs = np.where(mask == 0)

display_data[idxs[0], idxs[1]] = np.nan

im_output_fn = eye_id + '_all_wilcoxon_choroid_thickness_delta_reg_mm_outlierremoval.png'

title_name = ( eye_id +', significance of choroid thickness change; Wilcoxon signed-rank test') 
map_pValues(p_values)

In [ ]:
display_data = np.nanmean(r_maps, axis=0)

idxs = np.where(nan_map>=max_exclusion)
display_data[idxs] = np.nan
idxs = np.where(nan_map>=max_exclusion)
display_data[idxs] = np.nan

idxs = np.where(p_values>0.05)
display_data[idxs] = np.nan
title_name = ('Choroid areas of significant change, ' + eye_id +
              ', avg: ' + str(np.round(np.nanmean(display_data), 2)) +
              ', std: ' + str(np.round(np.nanstd(display_data), 2))  +
              ', max: ' + str(np.round(np.nanmax(display_data), 2)) + 
              ', min: ' + str(np.round(np.nanmin(display_data), 2))  
             ) 
map_significant_data(display_data, save_file=False, output_fpn=None)

In [ ]:
import matplotlib as mpl
maxmin_value = 10

fig = plt.figure(figsize=(8, 6), dpi=300)

d1 = np.nanmean(r_maps, axis=0)
idxs = np.where(nan_map>=max_exclusion)
d1[idxs] = np.nan
idxs = np.where(nan_map>=max_exclusion)
d1[idxs] = np.nan

idxs = np.where(p_values<0.05)
d1[idxs] = np.nan

d1 = d1.flatten()

d1[d1 >  maxmin_value] =  maxmin_value
d1[d1 < -maxmin_value] = -maxmin_value



d2 = np.nanmean(r_maps, axis=0)
idxs = np.where(nan_map>=max_exclusion)
d2[idxs] = np.nan
idxs = np.where(nan_map>=max_exclusion)
d2[idxs] = np.nan

d2[d2 > maxmin_value] =  maxmin_value
d2[d2< -maxmin_value] = -maxmin_value
idxs = np.where(p_values>=0.05)
d2[idxs] = np.nan

d2 = d2.flatten()

fig_hist =plt.hist((d1, d2) , 100, color=('grey', 'red'))
plt.title(eye_id +', choroid thickness change data distriburion', size=18)
plt.legend(['none significant data', 'statistically significant data'], 
           prop={'size':14}, frameon=False, loc='upper left')
plt.xlabel('μm, 100 bins', size = 16)
# plt.ylabel('my y label', size = 16)
plt.xlim([-3.5, 8])
plt.xticks(size = 16)
plt.yticks(size = 16)



In [ ]:
np.sum(~np.isnan(d2))/(np.sum(~np.isnan(d1))+np.sum(~np.isnan(d2)))

In [ ]:
display_data = np.nanmean(r_maps, axis=0)

left = display_data[0:256, 0:128][~np.isnan(display_data[0:256, 0:128])]

centre = display_data[50: 200, 192:320][~np.isnan(display_data[50: 200, 192:320])]

right = display_data[0:256, 384:512][~np.isnan(display_data[0:256, 384:512])]


plt.figure(figsize=(12, 8), dpi=80)
data = [left, centre, right]
fig7, ax7 = plt.subplots(figsize=(12, 8), dpi=80)
ax7.set_title('Effects of optical defocus on central (foveal) and peripheral retina')
ax7.boxplot(data)
plt.xticks([1, 2, 3], ['myopic defocus peripheral left', 'centre fovea area', 'hyperopic defocus peripheral right'])
plt.grid(axis='y')
plt.show()

In [ ]:
# sumarize residual data from areas and record in table, to csv



rscan_types_all = ['OD_residual', 'OS_residual', ]
column_names = ['px_id', 'eye_id', 
                'overall_mean_raw', 'overall_std_raw', 'overall_nonenans_raw',
                'overall_mean', 'overall_std', 'overall_nonenans',
                'perc_data_incl', 
                'lhs_area_mean', 'lhs_area_std', 'lhs_valid_data', 
                'centre_area_mean', 'centre_area_std', 'centre_valid_data',
                'rhs_area_mean', 'rhs_area_std', 'rhs_valid_data', 
               ]

def summary_area_of_interest(arr1, arr2, px_id, eye_id, column_names):

    df_new = pd.DataFrame(columns=column_names)
    df_new.loc[0,'px_id'] = px_id
    df_new.loc[0,'eye_id'] = eye_id
    df_new.loc[0,'overall_mean_raw'] = np.nanmean(arr1)
    df_new.loc[0,'overall_std_raw'] =  np.nanstd( arr1)                                         
    df_new.loc[0,'overall_nonenans_raw'] =  np.sum(~np.isnan(arr1))

    df_new.loc[0,'overall_mean'] = np.nanmean(arr2)
    df_new.loc[0,'overall_std'] =  np.nanstd( arr2)                                         
    df_new.loc[0,'overall_nonenans'] =  np.sum(~np.isnan(arr2))
    
    df_new.loc[0,'perc_data_incl'] = np.round(df_new.loc[0,'overall_nonenans'] / 
                                               df_new.loc[0,'overall_nonenans_raw'] * 
                                               100, 2)
    
    left = arr2[0:256, 0:128]
    df_new.loc[0,'lhs_area_mean'] = np.nanmean(left)
    df_new.loc[0,'lhs_area_std'] =  np.nanstd( left)                                         
    df_new.loc[0,'lhs_valid_data'] = np.round(np.sum(~np.isnan(arr2[0:256, 0:128])) / 
                                              np.sum(~np.isnan(arr1[0:256, 0:128])) *
                                              100, 2)
    centre = arr2[50: 200, 192:320]
    df_new.loc[0,'centre_area_mean'] = np.nanmean(centre)
    df_new.loc[0,'centre_area_std'] =  np.nanstd(centre)
    df_new.loc[0,'centre_valid_data'] = np.round(np.sum(~np.isnan(arr2[50: 200, 192:320])) / 
                                                 np.sum(~np.isnan(arr1[50: 200, 192:320])) *
                                                 100, 2)
    right = arr2[0:256, 384:512]
    df_new.loc[0,'rhs_area_mean'] = np.nanmean(right)
    df_new.loc[0,'rhs_area_std'] =  np.nanstd(right)
    df_new.loc[0,'rhs_valid_data'] = np.round(np.sum(~np.isnan(arr2[0:256, 384:512])) / 
                                              np.sum(~np.isnan(arr1[0:256, 384:512])) *
                                                 100, 2)

    return df_new# sumarize data from areas and record in table, to csv


In [ ]:
# sumarize data from areas and record in table, to csv
px_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 23, 26, ] # all px
# px_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,     17, 18, 20, 23, 26, ] # all px - 16 
eye_id = 'OD'
df_all = pd.DataFrame(columns=column_names)
    
for idx, px_id in enumerate(px_ids):
    
    # raw data
    residuals_fn1 = str(px_id) +'_' + eye_id + '_choroid_thickness_map_mm_raw.csv'
    fpn1 = os.path.join(residuals_data_fp, residuals_fn1)
    arr1 = pd.read_csv(fpn1, index_col=0).values.astype(float) 
    # filtered data 
    residuals_fn2 = str(px_id) +'_' + eye_id + '_choroid_thickness_map_mm_std_filter.csv'
    fpn2 = os.path.join(residuals_data_fp, residuals_fn2)
    arr2 = pd.read_csv(fpn2, index_col=0).values.astype(float)  
    # create summary of areas of interest 
    df_new = summary_area_of_interest(arr1, arr2, px_id, eye_id, column_names)  
    
    df_all = df_all.append(df_new)

output_fn = eye_id + '_residuals_different_areas_maxstd6_incl_RX_AL.csv'
df_all.to_csv(output_fn)    
df_all   

In [ ]:
print('overall mean: ', np.nanmean(df_all.overall_mean), 'overall std: ', np.nanstd(df_all.overall_mean))


print('left mean:    ',np.nanmean(df_all.lhs_area_mean), 'left std:    ',np.nanstd(df_all.lhs_area_mean))
print('centrel mean: ',np.nanmean(df_all.centre_area_mean), 'centrel std: ',np.nanstd(df_all.centre_area_mean))
print('right mean:   ',np.nanmean(df_all.rhs_area_mean), 'right std:   ',np.nanstd(df_all.rhs_area_mean))

data_test = np.nanmean(r_maps, axis=0)
idxs = np.where(nan_map>=max_exclusion)
data_test[idxs] = np.nan
idxs = np.where(nan_map>=max_exclusion)
data_test[idxs] = np.nan


print(np.nanmean(data_test))

In [ ]:
fp = r'E:\studyIII\OCT_final_data'

fn = '1_OS_choroid_thickness_map_mm_raw.csv'
fpn = os.path.join(fp, fn)
#         print(fpn)
r_map_sample = pd.read_csv(fpn, index_col=0).values.astype(float) 

In [ ]:
title_name = ('Residual data without filtering, ' + eye_id +
              ', avg: ' + str(np.round(np.nanmean(r_map_sample), 2)) +
              ', std: ' + str(np.round(np.nanstd(r_map_sample), 2))  +
              ', max: ' + str(np.round(np.nanmax(r_map_sample), 2)) + 
              ', min: ' + str(np.round(np.nanmin(r_map_sample), 2))  
             ) 
map_significant_data(r_map_sample
                     , save_file=False, output_fpn=None)